In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import mysql.connector
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut
from ipynb.fs.full.SqlQuery import AddLocalStoreProduct
from ipynb.fs.full.SqlQuery import Addlocalstors

In [14]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [15]:
def mytek_links():    
    list=[]
    web = requests.get('https://www.mytek.tn/')
    parse = BeautifulSoup(web.content, 'html.parser')
    for c in parse.find_all('div', class_ = 'sm_megamenu_col_2'):
        link= c.find('a', class_ ='sm_megamenu_nodrop')['href']
        info= c.find('span', class_ ='sm_megamenu_title_lv-3').text.strip()
        mytek_dict={
            'link':link,
            'info':info
        }
        list.append(mytek_dict)   
    return list

In [16]:
def mytek_det(url):
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'lxml')
    try:
        name= soup.find('h1', class_ ='page-title').span.text
    except:
        name = 'Unknown'
    try:
        brand = soup.find('div', class_='amshopby-option-link').a['title'].replace(' ','').strip()
    except:
        brand = 'Unknown'
    try:
        brand_logo = soup.find('div', class_='amshopby-option-link').img['src']
    except:
        brand_logo ='None'
    try:
        price= soup.find('span', class_ ='price-container price-final_price tax weee').span.span.text.replace('&nbsp;','')
    except:
        price = '0TND'
    try:
        stock = soup.find('div', class_ = 'product-info-stock-sku').div.span.text.strip()
    except:
        stock = 'épuise'
    try:
        sku = soup.find('div', class_ = 'value').text.strip()
    except:
        sku = 'None'
    try:    
        des = soup.find('div', class_ = 'product attribute overview').div.p.text.strip()
    except:
        des='None'

    magsin_list=[]
    fiche_tech=[]
    magasin, status= '',''
    try:
        local_store = soup.find('table', class_='tab_retrait_mag').find_all('tr')
        for l in local_store:
            if(l.text):
                text=l.text.split(':')
                mag_dict={
                        'magasin':text[0],
                        'status':text[1]
                       }
                magsin_list.append(mag_dict)
                text=[]
    except:
        magsin_list='None'
    try:
        fiche_technique= soup.find('table', class_='data table additional-attributes').find_all('tr')
        for f in fiche_technique:
                column1= f.find('th').text
                column2= f.find('td').text
                fiche_dict={
                   'label':column1,
                    'data':column2
                }
                fiche_tech.append(fiche_dict)
    except:
        fiche_tech=[]
    article ={
            'name':name,
            'brand': brand,
            'brand_logo':brand_logo,
            'price': price,
            'stock': stock,
            'sku': sku,
            'des': des,
            'local_store':magsin_list,
            'fiche_tech':fiche_tech
        }
    return article

In [17]:
def next_url(url, x):
    if(x == 1):
        url=url+'?p='+str(x)
    else:
        url=url[:-1]
        if(x>10):
            url=url[:-1]
        url=url+str(x)
    web = requests.get(url)
    parse = BeautifulSoup(web.content, 'html.parser')
    if(parse.find('div', class_='message info empty')):
        return False  
    
    return url

In [11]:
m_list=[{'link':'https://www.mytek.tn/maison-bricolage.html','info':'Maison |Brico & Animalerie'}]

In [12]:
#mycursor = mydb.cursor()
supCatId=0
list=m_list
for l in list:
    link = l['link']
    info= l['info']
    text = requests.get(link)
    soup = BeautifulSoup(text.content, 'html.parser')
    categorieAdd(info, link, 16)
    supCatId=getCagId(info,16)
    for x in range(1,53):
        link= next_url(link,x)
        print(link)
        if(not(link == False)):
            web = requests.get(link)
            soup = BeautifulSoup(web.content, 'html.parser')
            prod = soup.find_all('li', class_ = 'item product product-item')
            for p in prod:
                url = p.find('a', class_ = 'product photo product-item-photo')['href']
                try:
                    img=p.find('div',class_='image-product').a.span.span.img.get('src')
                except:
                    img='None'
                det= mytek_det(url)
                brandAdd(det['brand'], det['brand_logo'], 16)
                supBrandId=getBrandId(det['brand'],16)
                prodId=AddProduct(det['name'], url, det['sku'], det['des'], det['price'], det['stock'], 16, supBrandId, supCatId, img)
                if(det['local_store'] != 'None'):
                    Addlocalstors(det['local_store'], 16)
                    AddLocalStoreProduct(det['local_store'], det['price'], prodId, 16)
                if(det['fiche_tech']):
                    AddProductAttribut(det['fiche_tech'], prodId)
        else:
            break
#mycursor.close()

https://www.mytek.tn/maison-bricolage.html?p=1
https://www.mytek.tn/maison-bricolage.html?p=2
https://www.mytek.tn/maison-bricolage.html?p=3
https://www.mytek.tn/maison-bricolage.html?p=4
https://www.mytek.tn/maison-bricolage.html?p=5
https://www.mytek.tn/maison-bricolage.html?p=6
https://www.mytek.tn/maison-bricolage.html?p=7
https://www.mytek.tn/maison-bricolage.html?p=8
https://www.mytek.tn/maison-bricolage.html?p=9
https://www.mytek.tn/maison-bricolage.html?p=10
https://www.mytek.tn/maison-bricolage.html?p=11
https://www.mytek.tn/maison-bricolage.html?p=12
https://www.mytek.tn/maison-bricolage.html?p=13
https://www.mytek.tn/maison-bricolage.html?p=14
https://www.mytek.tn/maison-bricolage.html?p=15
https://www.mytek.tn/maison-bricolage.html?p=16
https://www.mytek.tn/maison-bricolage.html?p=17
https://www.mytek.tn/maison-bricolage.html?p=18
https://www.mytek.tn/maison-bricolage.html?p=19
https://www.mytek.tn/maison-bricolage.html?p=20
https://www.mytek.tn/maison-bricolage.html?p=21
h